<a href="https://colab.research.google.com/github/SanePai/Neural_Networks-Assignment-1/blob/master/Question_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd "/content/drive/My Drive/NNFL"
!ls

/content/drive/My Drive/NNFL
assignment1.pdf  data4.xlsx		   training_feature_matrix.xlsx
data2.xlsx	 test_feature_matrix.xlsx  training_output.xlsx
data3.xlsx	 test_output.xlsx


In [2]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg
import math

In [3]:
def sigmoid(n):
  '''returns the element wise sigmoid value in list'''
  return 1/(1+np.exp(-n))

df = pd.read_excel("data3.xlsx", header = None)
df = np.array(df)
m = len(df) #Total instances
col = np.ones((m,1))
df = np.concatenate((col,df), axis = 1)
np.random.shuffle(df)
print(m)

100


In [4]:
#Training set
X_train = df[0:50,:-1]
Y_train = df[0:50,-1]
Y_train -= 1
#Validation
X_val = df[50:60,:-1]
Y_val = df[50:60,-1]
Y_val -= 1
#Testing
X_test = df[60:,:-1]
Y_test = df[60:,-1]
Y_test -= 1

X = X_train.copy()
Y = Y_train.copy()

In [5]:
k = 1500 #Iterations
alpha_range = np.linspace(0.0001,0.001,10)
m = len(X) #Training instances
Y = Y_train.copy()
W_list = np.ones((k,5))
J_list = np.ones(k)
MSE_list = np.ones(k)
Y_mean = Y_train.mean()
Y_std = Y_train.std()

def hypothesis(w,X,m):
  wtx = np.ones(m)
  for i in range(0,m):
    wtx[i] = w[0]*X[i][0] + w[1]*X[i][1] + w[2]*X[i][2] + w[3]*X[i][3] + w[4]*X[i][4]
  h = sigmoid(wtx)
  return h

def cost(h,Y,m):
  J = 0
  for i in range(0,m):
    J += (Y[i]*math.log10(h[i]) + (1-Y[i])*(math.log10(1 - h[i])))
  J = J*(-1/m)
  return J

def update(w,alpha,m,X,Y,h):
  '''Updates the weights'''
  d0 = 0
  d1 = 0
  d2 = 0
  d3 = 0
  d4 = 0
  for i in range(0,m):
    d0 = d0 + ((h[i] - Y[i])*X[i][0])
  for i in range(0,m):
    d1 = d1 + ((h[i] - Y[i])*X[i][1])
  for i in range(0,m):
    d2 = d2 + ((h[i] - Y[i])*X[i][2])
    d3 = d3 + ((h[i] - Y[i])*X[i][3])
    d4 = d4 + ((h[i] - Y[i])*X[i][4])
    
  

  t0 = w[0] - alpha*d0
  t1 = w[1] - alpha*d1
  t2 = w[2] - alpha*d2
  t3 = w[3] - alpha*d3
  t4 = w[4] - alpha*d4
  
  w[0]=t0
  w[1]=t1
  w[2]=t2
  w[3]=t3
  w[4]=t4
  return w

def denormalize(data,avg,vari):
  '''Denormalize a list using the passed in mean and variation'''
  data1 = []
  for i in range(0,len(data)):
    data1.append((data[i]*vari) + avg)
  return data1

def calc_mse(m, h, y):
  '''Calculate the MSE for a given hypothesis and actual output'''
  add = 0
  for i in range(0,m):
    add = add + (h[i] - y[i])**2
  err = add/m
  return err

In [6]:
#Batch Gradient Descent
MSE_grid = []
for alpha in alpha_range:
  w = np.random.rand(5)
  W_list[0] = w
  for i in range(0,k):
    h = hypothesis(w,X,m)
    J_list[i] = cost(h,Y,m)
    w = update(w,alpha,m,X,Y,h)
    W_list[i][0] = w[0]
    W_list[i][1] = w[1]
    W_list[i][2] = w[2]
    W_list[i][3] = w[3]
    W_list[i][4] = w[4]
    MSE_list[i] = (calc_mse(m,denormalize(h,Y_mean,Y_std),denormalize(Y,Y_mean,Y_std)))
  MSE_grid.append(MSE_list[-1])
index = np.argmin(MSE_grid)
alpha = alpha_range[index]

#Final BGD
w = np.random.rand(5)
W_list[0] = w
for i in range(0,k):
  h = hypothesis(w,X,m)
  J_list[i] = cost(h,Y,m)
  w = update(w,alpha,m,X,Y,h)
  W_list[i][0] = w[0]
  W_list[i][1] = w[1]
  W_list[i][2] = w[2]
  MSE_list[i] = (calc_mse(m,denormalize(h,Y_mean,Y_std),denormalize(Y,Y_mean,Y_std)))

In [7]:
def hyp_refine(h,m):
  for i in range(0,m):
    if h[i] < 0.5:
      h[i] = 0
    elif h[i] >= 0.5:
      h[i] = 1
hyp_refine(h,m)  
S = (Y-h).tolist()
S.count(0)
print(S.count(0)/len(S)*100)

100.0


In [8]:
#Checking the accuracy of Validation set
m_val = len(X_val)
h_val = hypothesis(w,X_val,m_val)
hyp_refine(h_val,m_val)
S = (Y-h).tolist()
S.count(0)
print(S.count(0)/len(S)*100)

100.0


In [9]:
#Accuracies are the same. The model is robust.
#Using the hypothesis on the training data
m_test = len(X_test)
h_test = hypothesis(w,X_test,m_test)
hyp_refine(h_test,m_test)
TP = np.sum(h_test * Y_test)
FP = np.sum(h_test * (1-Y_test))
FN = np.sum((1-h_test) * Y_test)
TN = np.sum((1-h_test) * (1-Y_test))

sensitivity = TP/(TP + FN)
specificity = TN/(TN + FP)
accuracy = (TP + TN)*100/(TP + TN + FP + FN)
print("Sensitivity :",sensitivity)
print("Specificity :",specificity)
print("Accuracy    :" ,accuracy,"percent")

Sensitivity : 1.0
Specificity : 1.0
Accuracy    : 100.0 percent
